In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

# Load movie ratings dataset
ratings = pd.read_csv('/content/ratings.csv')

# Convert user IDs and movie IDs to sequential integers
ratings['userId'] = pd.Categorical(ratings['userId']).codes
ratings['movieId'] = pd.Categorical(ratings['movieId']).codes

# Split data into training and validation sets
mask = np.random.rand(len(ratings)) < 0.8
train = ratings[mask]
val = ratings[~mask]

# Define model architecture
num_users = len(ratings['userId'].unique())
num_movies = len(ratings['movieId'].unique())

user_input = Input(shape=(1,))
user_embedding = Embedding(num_users, 32)(user_input)
user_embedding = Flatten()(user_embedding)

movie_input = Input(shape=(1,))
movie_embedding = Embedding(num_movies, 32)(movie_input)
movie_embedding = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_embedding, movie_embedding])
output = Dense(1)(dot_product)

model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model
model.fit(x=[train['userId'], train['movieId']], y=train['rating'], batch_size=64, epochs=10, validation_data=([val['userId'], val['movieId']], val['rating']))

# Generate movie recommendations for a user
user_id = 200
user_movies = ratings[ratings['userId'] == user_id]['movieId'].unique()
user_ratings = np.zeros(num_movies)
user_ratings[user_movies] = 1

movie_scores = model.predict([np.array([user_id]*num_movies), np.arange(num_movies)])
movie_ids = np.argsort(-movie_scores.flatten())[:10]
recommended_movies = pd.read_csv('/content/movies.csv')
recommended_movies = recommended_movies[recommended_movies['movieId'].isin(movie_ids)]
print(recommended_movies[['title', 'genres']])

Epoch 1/10
1262/1262 [==============================] - 13s 9ms/step - loss: 7.2559 - val_loss: 1.7703
Epoch 2/10
1262/1262 [==============================] - 10s 8ms/step - loss: 1.1724 - val_loss: 1.1053
Epoch 3/10
1262/1262 [==============================] - 9s 7ms/step - loss: 0.8048 - val_loss: 0.9965
Epoch 4/10
1262/1262 [==============================] - 8s 7ms/step - loss: 0.6572 - val_loss: 0.9575
Epoch 5/10
1262/1262 [==============================] - 9s 7ms/step - loss: 0.5391 - val_loss: 0.9551
Epoch 6/10
1262/1262 [==============================] - 9s 7ms/step - loss: 0.4396 - val_loss: 0.9630
Epoch 7/10
1262/1262 [==============================] - 9s 7ms/step - loss: 0.3588 - val_loss: 0.9832
Epoch 8/10
1262/1262 [==============================] - 9s 7ms/step - loss: 0.2956 - val_loss: 1.0014
Epoch 9/10
1262/1262 [==============================] - 9s 7ms/step - loss: 0.2476 - val_loss: 1.0268
Epoch 10/10
304/304 [==============================] - 0s 1ms/step
             

In [16]:
user_id = 400
user_movies = ratings[ratings['userId'] == user_id]['movieId'].unique()
user_ratings = np.zeros(num_movies)
user_ratings[user_movies] = 1

movie_scores = model.predict([np.array([user_id]*num_movies), np.arange(num_movies)])
movie_ids = np.argsort(-movie_scores.flatten())[:10]
recommended_movies = pd.read_csv('/content/movies.csv')
recommended_movies = recommended_movies[recommended_movies['movieId'].isin(movie_ids)]
pd.DataFrame(recommended_movies[['title', 'genres']])

304/304 [==============================] - 0s 1ms/step


,title,genres
563,"Alphaville (Alphaville, une étrange aventure d...",Drama|Mystery|Romance|Sci-Fi|Thriller
1595,Charlotte's Web (1973),Animation|Children
1954,Hideous Kinky (1998),Drama
2502,Blood Feast (1963),Horror
4373,Treasure Island (1950),Adventure|Children
4453,Eve of Destruction (1991),Action|Sci-Fi|Thriller
